In [ ]:
import os
import numpy as np
from numpy import random
import matplotlib
import pandas as pd
import pickle
import time
matplotlib.use('Tkagg')
import random as rd
from matplotlib import pyplot as plt
from keras import metrics

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
import tensorflow as tf
#%matplotlib inline 
#%matplotlib notebook

In [ ]:
plt.rc('font', size=14)
plt.rc('font', family='Arial')
plt.rc('mathtext', fontset='custom')
plt.rc('mathtext', rm='Arial')
plt.rc('mathtext', sf='Arial')
plt.rc('mathtext', it='Arial:italic')
plt.rc('mathtext', bf='Arial:bold')
plt.rc('lines', linewidth=1)
plt.rc('axes', linewidth=2)

In [ ]:
input_set = pd.read_csv("input.txt",dtype = np.float32, header = None)

In [ ]:
output_T = pd.read_csv("output_T.txt",header = None)

In [ ]:
output_S = pd.read_csv("output_S.txt",header = None)

In [ ]:
input_tr = input_set.sample(frac=0.8, random_state=25)
input_te = input_set.drop(input_tr.index)

In [ ]:
output_T_tr = output_T.sample(frac=0.8, random_state=25)
output_T_te = output_T.drop(output_T_tr.index)

In [ ]:
output_S_tr = output_S.sample(frac=0.8, random_state=25)
output_S_te = output_S.drop(output_S_tr.index)

In [ ]:
### hyperparameter settings and other constants
batch_size = 32
epochs = 20
d1 = 1024
d2 = 256
alpha_adam = 0.001
rho1 = 0.99
rho2 = 0.999
### end hyperparameter settings


In [ ]:
def evaluate_model(Xs, Ys, model):
    (loss, accuracy) = model.evaluate(Xs, Ys)
    return (loss, accuracy)

In [ ]:
def plot_fig(historysgd1,loss_te_sgd1,accuracy_te_sgd1,name):
    matplotlib.rcParams['figure.figsize'] = [12, 4]
    fig, axs = plt.subplots(1,2)
    axs[0].plot(historysgd1.history['loss'],linewidth = 2)
    axs[0].plot(historysgd1.history['val_loss'],linewidth = 2)
    axs[0].plot([loss_te_sgd1 for i in range(epochs)],linewidth = 2)
    axs[0].title.set_text('Loss')
    axs[0].set(xlabel= 'Epochs', ylabel = 'Loss')
    axs[0].legend(['Training data', 'Validation data','Test data'])

    axs[1].plot(100 - np.array(historysgd1.history['mean_absolute_percentage_error']),linewidth = 2)
    axs[1].plot(100 - np.array(historysgd1.history['val_mean_absolute_percentage_error']),linewidth = 2)
    axs[1].plot([100 - accuracy_te_sgd1 for i in range(epochs)],linewidth = 2)
    axs[1].title.set_text('Accuracy')
    axs[1].set(xlabel= 'Epochs', ylabel = 'Accuracy %')
    axs[1].legend(['Training data', 'Validation data','Test data'])
    plt.tight_layout()
    plt.savefig(name,dpi=1000);
    plt.show()

In [ ]:
def train_fully_connected_adam(Xs, Ys, d1, d2, alpha, rho1, rho2, B, epochs):
    n,i = Xs.shape
    model = tf.keras.Sequential()
    model.add(Dense(26))
    model.add(Dense(d1, activation='relu'))
    model.add(Dense(d2, activation='relu'))
    model.add(Dense(26))
    model.compile(loss=tf.keras.losses.MeanSquaredError(), 
                  optimizer=tf.keras.optimizers.Adam(learning_rate=alpha, beta_1=rho1, beta_2=rho2, epsilon=1e-07), 
                  metrics=[metrics.mean_absolute_percentage_error])
    history = model.fit(Xs, Ys, batch_size=B, epochs=epochs, validation_split=0.1)
    return (model, history)

In [ ]:
start = time.time()
modeladam, historyadam = train_fully_connected_adam(input_tr, output_T_tr, d1, d2, alpha_adam, rho1, rho2, batch_size, epochs)
print(time.time() - start)

In [ ]:
loss_te_adam, accuracy_te_adam = evaluate_model(input_te, output_T_te, modeladam)

In [ ]:
plot_fig(historyadam, loss_te_adam, accuracy_te_adam, name = "NN_T")

In [ ]:
start = time.time()
modeladamS, historyadamS = train_fully_connected_adam(input_tr, output_S_tr/1e7, d1, d2, alpha_adam, rho1, rho2, batch_size, epochs)
print(time.time() - start)

In [ ]:
loss_te_adamS, accuracy_te_adamS = evaluate_model(input_te, output_S_te/1e7, modeladamS)

In [ ]:
plot_fig(historyadamS, loss_te_adamS, accuracy_te_adamS, name = "NN_S")

In [ ]:
def Rand_properties():
    matRan = [rd.randint(0,1) for i in range(26)];
    return matRan

In [ ]:
def PredSample_gen(sd, input_set, NSample):
    rd.seed(sd)
    predict_set = np.zeros([NSample,26])
    iNum = 0
    while (iNum < NSample):
        ran = Rand_properties()
        if ((ran == input_set).all(1).any() == False):
            predict_set[iNum] = ran
            iNum += 1
    return predict_set
        

In [ ]:
predict_set = PredSample_gen(6532, input_set, 100000)

In [ ]:
start = time.time()
p  = modeladam.predict(predict_set)
print(time.time() - start)

In [ ]:
start = time.time()
pS  = modeladamS.predict(predict_set)
print(time.time() - start)

In [ ]:
output_T_p = pd.read_csv("output_T_p.txt",header = None)

In [ ]:
output_S_p = pd.read_csv("output_S_p.txt",header = None)

In [ ]:
color = ["red", "blue", "cyan", "purple", "green"]
label = ["New 1","New 2","New 3","New 4","New 5"]
fig, axs = plt.subplots(1,2)
for i in range(5):
    #axs[0].scatter(np.arange(1,27), output_T_p.T[i], marker = ".", edgecolor = "black", color = color[i], label = label[i])
    #axs[0].scatter(np.arange(1,27),p[:5][i], marker = "*", edgecolor = "black",alpha = 0.5, color = color[i])
    axs[1].scatter(np.arange(1,27),output_S_p.T[i], edgecolor = "black",marker = ".", color = color[i], label = label[i])
    axs[1].scatter(np.arange(1,27),(pS[:5]*1e7)[i], edgecolor = "black",marker = "*", alpha = 0.5, color = color[i])
#axs[1].legend(ncol = 5, bbox_to_anchor=(0., 1.2))
axs[0].scatter(np.arange(1,27), output_T_p.T[i], marker = ".", edgecolor = "black", color = "black", label = "FEA")
axs[0].scatter(np.arange(1,27),p[:5][i], marker = "*", edgecolor = "black",alpha = 0.5, color = "black", label = "DNN")
axs[0].legend(ncol = 5, bbox_to_anchor=(1.0, 1.2))
plt.tight_layout()
plt.savefig("pre_legend2.png",dpi=1000);
plt.show()